# 영상의 기하학적 변환

In [2]:
import cv2
import numpy as np

### 회전 변환

In [4]:
src.shape

(480, 640, 3)

In [6]:
src = cv2.imread('./data/tekapo.bmp')

h, w = src.shape[0], src.shape[1]

cp = h/2, w/2

M = cv2.getRotationMatrix2D(cp, 20, 0.5)

dst = cv2.warpAffine(src, M, (0,0))

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [9]:
# 참고
dst1 = cv2.rotate(src, cv2.ROTATE_90_CLOCKWISE)
dst2 = cv2.rotate(src, cv2.ROTATE_90_COUNTERCLOCKWISE)
cv2.imshow('src', src)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

### 대칭 변환

In [16]:
src = cv2.imread('./data/eastsea.bmp')

cv2.imshow('src', src)

for flip_code in [1, 0, -1]:
    dst = cv2.flip(src, flip_code)
    desc = "flip code: %d"% flip_code
    cv2.putText(dst, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 1, cv2.LINE_AA)
    cv2.imshow('dst', dst)
    cv2.waitKey()


cv2.destroyAllWindows()

# 투시변환

In [17]:
import sys

In [23]:
cnt = 0
src_pts = np.zeros([4,2], dtype=np.float32)

def on_mouse(event, x, y, flags, param):
    global src_pts, cnt
    if event == cv2.EVENT_LBUTTONDOWN:
        if cnt < 4:
            src_pts[cnt, :] = np.array([x, y], dtype=np.float32)
            cnt += 1
            cv2.circle(src, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow('src', src)
            
        if cnt == 4:
            w, h = (200, 300)
            dst_pts = np.array([[0,0], [w-1, 0], [w-1, h-1], [0, h-1]], dtype=np.float32)
            M = cv2.getPerspectiveTransform(src_pts, dst_pts)
            dst = cv2.warpPerspective(src, M, (w, h))
            cv2.imshow('dst', dst)

src = cv2.imread('./data/card.bmp')

if src is None:
    print('Image load failed')
    sys.exit()

cv2.imshow('src', src)
cv2.setMouseCallback('src', on_mouse)

cv2.waitKey()
cv2.destroyAllWindows()

# 에지 검출과 응용

### 마스크 기반 에지 검출 - 소벨 마스크

In [25]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print("image load failed")
    sys.exit()

mx = np.array(
     [[-1, 0, 1],
      [-2, 0, 2],
      [-1, 0, 1]], dtype=np.float32)
     

my = np.array(
     [[-1, -2, -1],
      [0, 0, 0],
      [1, 2, 1]], dtype=np.float32)
     

dx = cv2.filter2D(src, -1, mx, delta=128)
dy = cv2.filter2D(src, -1, my, delta=128)

# dx = cv2.Sobel(src, -1, 1, 0, delta=128)    
# dy = cv2.Sobel(src, -1, 0, 1, delta=128)    

cv2.imshow('src', src)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)
cv2.waitKey()
cv2.destroyAllWindows()

In [48]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print("image load failed")
    sys.exit()
    
dx = cv2.Sobel(src, cv2.CV_32FC1, 1, 0)    
dy = cv2.Sobel(src, cv2.CV_32FC1, 0, 1)    

fmag = cv2.magnitude(dx, dy)
mag = np.clip(fmag, 0, 255).astype(np.uint8)

_, edge = cv2.threshold(mag, 160, 255, cv2.THRESH_BINARY)

cv2.imshow('src', src)
cv2.imshow('mag', mag)
cv2.imshow('edge', edge)
cv2.waitKey()
cv2.destroyAllWindows()

### 캐니 에즈 검출기

In [53]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print("image load failed")
    sys.exit()
    
dst1 = cv2.Canny(src, 50, 100)    
dst2 = cv2.Canny(src, 50, 150)

cv2.imshow('src', src)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

### 허프 변환 직선 검출

In [68]:
import math
src = cv2.imread('./data/building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print("image load failed")
    sys.exit()
    
edge = cv2.Canny(src, 50, 150) 

rho = 1
theta = math.pi/180

lines = cv2.HoughLines(edge, rho, theta, 250)

dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

if lines is not None:
    for i in range(lines.shape[0]):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        cos_t = math.cos(theta)
        sin_t = math.sin(theta)
        
        x0, y0 = rho*cos_t, rho * sin_t
        alpha = 1000
        
        pt1 = (int(x0 -alpha*sin_t), int(y0+alpha*cos_t))
        pt2 = (int(x0 +alpha*sin_t), int(y0-alpha*cos_t))
        
        cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)


cv2.imshow('src', src)
cv2.imshow('edge', edge)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [82]:
src = cv2.imread('./data/building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print("image load failed")
    sys.exit()
    
edge = cv2.Canny(src, 50, 150) 

dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

rho = 1
theta = math.pi/180

lines = cv2.HoughLinesP(edge, rho, theta, 160, minLineLength=50, maxLineGap=5)

if lines is not None:
    for i in range(lines.shape[0]):
        pt1 = (lines[i][0][0], lines[i][0][1])
        pt2 = (lines[i][0][2], lines[i][0][3])
        cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)
        
        
cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()        

### 허프 변환 원 검출

In [102]:
src = cv2.imread('./data/coins.png', cv2.IMREAD_GRAYSCALE)

if src is None:
    print("image load failed")
    sys.exit()

blurred = cv2.blur(src, (3,3))    
circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=150, param2=30 )

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

if circles is not None:
    for i in range(circles.shape[1]):
        cx, cy, radius = circles[0][i]
        cv2.circle(dst, (np.uint32(cx), np.uint32(cy)), np.uint32(radius), (0, 0, 255), 2)
        
cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()   